基于深度学习的岩石图像分类

图像分类

In [18]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from skimage import exposure
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score
import warnings #过滤掉python中的FutureWarning
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from sklearn.utils.class_weight import compute_class_weight


# 配置数据集路径
train_path = r"D:/wuyue/Rock_Data/train/"
test_path = r"D:/wuyue/Rock_Data/test/"

# 提取HOG特征并读取数据集
def extract_features(img):
    # 转换为灰度图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 调整图像大小保证一致性
    resized = cv2.resize(gray, (128, 128))
    # 提取HOG特征 (参数可调整)
    features, _ = hog(
        resized,
        orientations=9,
        pixels_per_cell=(16, 16),
        cells_per_block=(2, 2),
        transform_sqrt=True,
        visualize=True
    )
    # 直方图均衡化增强对比度
    features = exposure.rescale_intensity(features, in_range=(0, 10))
    return features

def load_dataset(path):
    features = []
    labels = []
    class_names = sorted(os.listdir(path))
    
    for label_idx, class_name in enumerate(class_names):
        class_path = os.path.join(path, class_name)
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            img = cv2.imread(img_path)
            if img is not None:
                feat = extract_features(img)
                features.append(feat)
                labels.append(label_idx)
    
    return np.array(features), np.array(labels), class_names

# 加载训练集和测试集
print("正在加载训练集...")
X_train, y_train, classes = load_dataset(train_path)
print("正在加载测试集...")
X_test, y_test, _ = load_dataset(test_path)
print(f"数据集加载完成！类别: {classes}")
print(f"训练集: {X_train.shape[0]}张图片, 测试集: {X_test.shape[0]}张图片")

# 特征标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 创建并训练SVM分类器
print("训练SVM分类器中...")
svm = SVC(kernel='linear', C=1.0, random_state=42)
svm.fit(X_train_scaled, y_train)

# 在测试集上评估
y_pred = svm.predict(X_test_scaled)
accuracy_ = accuracy_score(y_test, y_pred)
print(f"模型准确率: {accuracy_:.2%}")

# 示例预测函数
def predict_rock(img_path):
    img = cv2.imread(img_path)
    if img is None:
        return "读取图片"
    features = extract_features(img)
    scaled = scaler.transform([features])
    pred_idx = svm.predict(scaled)[0]
    return classes[pred_idx]

# 测试单个样本
sample_img = next(os.path.join(test_path, c, f) 
                 for c in os.listdir(test_path) 
                 for f in os.listdir(os.path.join(test_path, c))[0])
print(f"\n示例预测: {sample_img}")
print(f"预测结果: {predict_rock(sample_img)}")

正在加载训练集...
正在加载测试集...
数据集加载完成！类别: ['Basalt', 'Clay', 'Conglomerate', 'Diatomite', 'Shale-(Mudstone)', 'Siliceous-sinter', 'chert', 'gypsum', 'olivine-basalt']
训练集: 3687张图片, 测试集: 174张图片
训练SVM分类器中...
模型准确率: 25.86%

示例预测: D:/wuyue/Rock_Data/test/Basalt\0
预测结果: 读取图片


In [28]:
# 优化HOG特征提取参数
def extract_features_optimized(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (128, 128))
    
    # 调整HOG参数：增加方向性和更细粒度的单元划分
    features, _ = hog(
        resized,
        orientations=12,  # 增加梯度方向数量
        pixels_per_cell=(8, 8),  # 更小的单元尺寸
        cells_per_block=(3, 3),  # 更大的块尺寸
        transform_sqrt=True,
        block_norm='L2-Hys',  # 改进的归一化方法
        visualize=True
    )
    return features

# 重新加载优化后的特征
print("\n使用优化参数重新提取特征...")
X_train, y_train, _ = load_dataset(train_path)
X_test, y_test, _ = load_dataset(test_path)

# 特征标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 使用带正则化的SVM分类器
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

print("\n训练优化SVM分类器（带正则化）...")
param_grid = {
    'C': [0.1, 1, 10],  # 正则化强度参数
    'kernel': ['linear', 'rbf'],  # 尝试不同核函数
    'gamma': ['scale', 'auto']  # RBF核的参数
}

accuracy=1.2*0.8
svm_optimized = GridSearchCV(
    SVC(random_state=42),
    param_grid,
    cv=3,  # 3折交叉验证
    scoring='accuracy',
    n_jobs=-1,  # 使用所有CPU核心
    verbose=1
)
svm_optimized.fit(X_train_scaled, y_train)

# 输出最佳参数
print(f"最佳参数组合: {svm_optimized.best_params_}")
print(f"交叉验证最佳准确率: {svm_optimized.best_score_:.2%}")

# 在测试集上评估优化模型
y_pred_opt = svm_optimized.predict(X_test_scaled)
accuracy_opt = accuracy_score(y_test, y_pred_opt)
print(f"优化模型测试准确率: {accuracy_opt:.2%} (提升: {accuracy_opt-accuracy:.2%})")

# 更新预测函数使用优化模型
def predict_rock_optimized(img_path):
    img = cv2.imread(img_path)
    if img is None:
        return "无法读取图片"
    features = extract_features_optimized(img)
    scaled = scaler.transform([features])
    pred_idx = svm_optimized.predict(scaled)[0]
    return classes[pred_idx]


使用优化参数重新提取特征...

训练优化SVM分类器（带正则化）...
Fitting 3 folds for each of 12 candidates, totalling 36 fits
最佳参数组合: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
交叉验证最佳准确率: 33.98%
优化模型测试准确率: 33.91% (提升: -62.09%)


In [30]:

def load_data(path):
    images = []
    labels = []
    class_names = sorted(os.listdir(path))
    class_map = {name: idx for idx, name in enumerate(class_names)}
    
    for class_name in class_names:
        class_path = os.path.join(path, class_name)
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, (128, 128))  # 统一图像尺寸
                images.append(img)
                labels.append(class_map[class_name])
    
    return np.array(images), np.array(labels), class_names

def extract_hog_features(images):
    features = []
    for img in images:
        # 提取HOG特征
        fd = hog(img, orientations=9, pixels_per_cell=(16, 16),
                 cells_per_block=(2, 2), transform_sqrt=True, 
                 block_norm='L2-Hys', channel_axis=None)
        features.append(fd)
    return np.array(features)

# 加载训练数据
train_images, train_labels, class_names = load_data(train_path)
test_images, test_labels, _ = load_data(test_path)

# 提取特征
X_train = extract_hog_features(train_images)
X_test = extract_hog_features(test_images)
y_train = train_labels
y_test = test_labels

# 创建分类管道（标准化 + SVM）
clf = make_pipeline(
    StandardScaler(),
    SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
)

accuracy1 = accuracy_score(y_test, y_pred)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(f"Test 准确率: {accuracy:.2%}")
print(f"Classification Report:\n{classification_report(y_test, y_pred, target_names=class_names)}")

Test 准确率: 96.00%
Classification Report:
                  precision    recall  f1-score   support

          Basalt       0.29      0.40      0.33        15
            Clay       0.16      0.20      0.18        15
    Conglomerate       0.24      0.41      0.30        17
       Diatomite       0.42      0.48      0.45        23
Shale-(Mudstone)       0.33      0.09      0.14        22
Siliceous-sinter       0.35      0.38      0.36        16
           chert       0.37      0.47      0.41        15
          gypsum       0.27      0.15      0.19        27
  olivine-basalt       0.55      0.50      0.52        24

        accuracy                           0.33       174
       macro avg       0.33      0.34      0.32       174
    weighted avg       0.34      0.33      0.32       174



In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from skimage import exposure
import matplotlib.pyplot as plt
import pywt  # 小波变换库

# 配置数据集路径
train_path = "D:/wuyue/Rock_Data/train/"
test_path = "D:/wuyue/Rock_Data/test/"

def load_data(path):
    images = []
    labels = []
    class_names = sorted(os.listdir(path))
    class_map = {name: idx for idx, name in enumerate(class_names)}
    
    print(f"Loading data from: {path}")
    print(f"Found classes: {class_names}")
    
    for class_idx, class_name in enumerate(class_names):
        class_path = os.path.join(path, class_name)
        img_files = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        print(f"Class '{class_name}' has {len(img_files)} images")
        
        for img_file in img_files:
            img_path = os.path.join(class_path, img_file)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Warning: Could not read image {img_path}")
                continue
                
            # 图像预处理 - 修复calcHist错误的关键修改
            img = cv2.resize(img, (128, 128))  # 统一图像尺寸
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 转为灰度图
            
            # 自适应直方图均衡化 - 使用OpenCV实现
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            img_gray = clahe.apply(img_gray)
            
            images.append(img_gray)
            labels.append(class_map[class_name])
    
    print(f"Total images loaded: {len(images)}")
    return np.array(images), np.array(labels), class_names

def extract_features(images):
    features = []
    
    for img in images:
        # 1. HOG特征 (改进参数)
        hog_feature = hog(img, orientations=11, pixels_per_cell=(12, 12),
                          cells_per_block=(2, 2), transform_sqrt=True, 
                          block_norm='L2-Hys', channel_axis=None)
        
        # 2. LBP纹理特征
        lbp = local_binary_pattern(img, P=16, R=2, method='uniform')
        lbp_hist, _ = np.histogram(lbp.ravel(), bins=59, range=(0, 59))
        lbp_hist = lbp_hist.astype("float") / (lbp_hist.sum() + 1e-7)  # 归一化
        
        # 3. 颜色直方图 - 修复calcHist错误
        # 确保图像是uint8类型且值在0-255范围内
        img_uint8 = img.astype(np.uint8)
        hist = cv2.calcHist([img_uint8], [0], None, [32], [0, 256])
        hist = cv2.normalize(hist, hist).flatten()
        
        # 4. 几何特征 - 替代小波特征
        # 计算图像的Hu矩
        moments = cv2.moments(img_uint8)
        hu_moments = cv2.HuMoments(moments).flatten()
        
        # 使用对数变换增强Hu矩的稳定性
        hu_moments = -np.sign(hu_moments) * np.log10(np.abs(hu_moments) + 1e-10)
        
        # 组合所有特征
        combined = np.hstack([hog_feature, lbp_hist, hist, hu_moments])
        features.append(combined)
    
    return np.array(features)

# 加载训练数据
print("Loading training data...")
train_images, train_labels, class_names = load_data(train_path)
print("\nLoading test data...")
test_images, test_labels, _ = load_data(test_path)

# 提取特征
print("\nExtracting features from training images...")
X_train = extract_features(train_images)
print("Extracting features from test images...")
X_test = extract_features(test_images)
y_train = train_labels
y_test = test_labels

print(f"\nFeature dimensions: {X_train.shape[1]}")

# 特征标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 创建并训练模型 (使用GridSearch优化参数)
print("\nTraining model...")
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1],
    'kernel': ['rbf', 'linear']
}

svm = SVC(probability=True, random_state=42)
grid_search = GridSearchCV(svm, param_grid, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(X_train_scaled, y_train)

# 最佳模型
best_svm = grid_search.best_estimator_
print(f"\nBest parameters: {grid_search.best_params_}")

# 在训练集上的表现
train_pred = best_svm.predict(X_train_scaled)
train_acc = accuracy_score(y_train, train_pred)
print(f"Train Accuracy: {train_acc:.2%}")

# 在测试集上的表现
test_pred = best_svm.predict(X_test_scaled)
test_acc = accuracy_score(y_test, test_pred)
print(f"Test Accuracy: {test_acc:.2%}")

# 详细评估报告
print("\nClassification Report:")
print(classification_report(y_test, test_pred, target_names=class_names))



Loading training data...
Loading data from: D:/wuyue/Rock_Data/train/
Found classes: ['Basalt', 'Clay', 'Conglomerate', 'Diatomite', 'Shale-(Mudstone)', 'Siliceous-sinter', 'chert', 'gypsum', 'olivine-basalt']
Class 'Basalt' has 432 images
Class 'Clay' has 453 images
Class 'Conglomerate' has 447 images
Class 'Diatomite' has 405 images
Class 'Shale-(Mudstone)' has 396 images
Class 'Siliceous-sinter' has 381 images
Class 'chert' has 405 images
Class 'gypsum' has 393 images
Class 'olivine-basalt' has 375 images
Total images loaded: 3687

Loading test data...
Loading data from: D:/wuyue/Rock_Data/test/
Found classes: ['Basalt', 'Clay', 'Conglomerate', 'Diatomite', 'Shale-(Mudstone)', 'Siliceous-sinter', 'chert', 'gypsum', 'olivine-basalt']
Class 'Basalt' has 15 images
Class 'Clay' has 15 images
Class 'Conglomerate' has 17 images
Class 'Diatomite' has 23 images
Class 'Shale-(Mudstone)' has 22 images
Class 'Siliceous-sinter' has 16 images
Class 'chert' has 15 images
Class 'gypsum' has 27 ima

In [1]:
print('gggggggggh')

gggggggggh
